In [8]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters langchain-chroma langchain-core langchain-upstage
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [60]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # Chunk 한개가 가질 수 있는 토큰 수
    chunk_overlap=300, # Chunk 를 겹치게 자르는, 1 - 3, 다음은 2 - 4 이런식으로
)

# 문서를 가져와서,
loader = Docx2txtLoader('./tax.docx')

# Chunk 단위로 문서를 쪼갠다
document_list = loader.load_and_split(
    text_splitter=text_splitter
)

In [62]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(model='gpt-4o')

embedding = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

- Chroma : Vector in-memory Database

In [1]:
%pip install -qU langchain langchain-pinecone langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [45]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
index_name = "tax-index"

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, # Chunk 한개가 가질 수 있는 토큰 수
    chunk_overlap=100, # Chunk 를 겹치게 자르는, 1 - 3, 다음은 2 - 4 이런식으로
)

chunked_documents = text_splitter.split_documents(document_list)

database = PineconeVectorStore.from_documents(
    documents=[],
    embedding=embedding,
    index_name=index_name
)

batch_size = 100
for i in range(0, len(chunked_documents), batch_size):
    print(f'index: {i}, batch size : {batch_size}')
    batch = chunked_documents[i: i + batch_size]
    database.add_documents(batch)

index: 0, batch size : 100
index: 100, batch size : 100
index: 200, batch size : 100
index: 300, batch size : 100
index: 400, batch size : 100


- 요런 과정으로 파인콘의 인덱스를 생성하지만, 파인콘 콘솔을 통해서도 생성할 수 있음.
```python
from pinecone import Pinecone, ServerlessSpec

index_name = "tax-index"  # change if desired

pc = Pinecone(api_key=pinecone_api_key)
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)
```

In [47]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrived_docs = database.similarity_search(query, k = 3)
print(retrived_docs)

[Document(id='ada43e6d-a5b8-429a-9786-324d4aa1fd13', metadata={'source': './tax.docx'}, page_content='1. 이자소득에 대해서는 다음에 규정하는 세율\n\n가. 삭제<2017. 12. 19.>\n\n나. 비영업대금의 이익에 대해서는 100분의 25. 다만, 「온라인투자연계금융업 및 이용자 보호에 관한 법률」에 따라 금융위원회에 등록한 온라인투자연계금융업자를 통하여 지급받는 이자소득에 대해서는 100분의 14로 한다.\n\n다. 제16조제1항제10호에 따른 직장공제회 초과반환금에 대해서는 기본세율\n\n라. 그 밖의 이자소득에 대해서는 100분의 14\n\n2. 배당소득에 대해서는 다음에 규정하는 세율\n\n가. 제17조제1항제8호에 따른 출자공동사업자의 배당소득에 대해서는 100분의 25\n\n나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 

In [51]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [57]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs={"prompt" : prompt}
)

ai_message = qa_chain.invoke({"query" : query})
print(ai_message)

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?', 'result': '연봉 5천만원인 직장인의 소득세는 근로소득 기본세율에 따라 계산됩니다. 기본세율은 소득 구간에 따라 차등 적용됩니다. 정확한 소득세 금액은 구체적인 소득세율표와 공제 항목을 참조하여 산출해야 합니다.'}
